In [2]:
import pandas as pd
from plotly.offline import iplot
import plotly.graph_objs as go
import numpy as np
from sklearn import metrics
import math, os
import matplotlib.pyplot as plt
from collections import defaultdict
from functools import partial
from itertools import repeat
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import roc_curve, auc
from keras.utils import to_categorical
from keras import models, layers, metrics
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Dropout, Activation
import keras.backend as K

Using TensorFlow backend.


In [3]:
# returns dataframe
# params: full -> True = full dataset, False = first file

def grab_data(full, path):
    if full:
        return pd.concat([pd.read_csv(path+file, index_col=False) for file in os.listdir(path)])
    else:
        for file in os.listdir(path):
            if "10_1_" not in file: continue
            return pd.read_csv(path+file, index_col=False)

In [87]:
# import data

nBodies = 3
workDir = "/Users/brandonmanley/Documents/nBody/data"
dataPath = "/Users/brandonmanley/Documents/nBody/data/brutusSim/"+str(nBodies)+"body/"
        
df = grab_data(True, dataPath)
df.head()

,file,eventID,m1,m2,m3,x1,x2,x3,y1,y2,...,xf3,yf1,yf2,yf3,dxf1,dxf2,dxf3,dyf1,dyf2,dyf3
0,1,10001,5.702832,58.638287,54.862347,-0.429122,-3.287465,2.112128,-8.012798,9.542259,...,2.111637,-8.009622,9.541494,3.759572,0.321621,-0.577427,-0.127013,0.814151,-0.197060,0.279734
1,1,10002,5.702832,58.638287,54.862347,-0.429122,-3.287465,2.112128,-8.012798,9.542259,...,2.111136,-8.006437,9.540719,3.760669,0.321816,-0.575079,-0.129543,0.816311,-0.199630,0.282257
2,1,10003,5.702832,58.638287,54.862347,-0.429122,-3.287465,2.112128,-8.012798,9.542259,...,2.110625,-8.003244,9.539934,3.761777,0.322011,-0.572729,-0.132075,0.818473,-0.202200,0.284779
3,1,10004,5.702832,58.638287,54.862347,-0.429122,-3.287465,2.112128,-8.012798,9.542259,...,2.110104,-8.000043,9.539140,3.762894,0.322205,-0.570379,-0.134607,0.820635,-0.204770,0.287301
4,1,10005,5.702832,58.638287,54.862347,-0.429122,-3.287465,2.112128,-8.012798,9.542259,...,2.109573,-7.996833,9.538335,3.764021,0.322400,-0.568027,-0.137141,0.822798,-0.207339,0.289821


In [88]:
# INPUT: want mass, x/y pos, dx/dy for n bodies -> total input is n*5 + 1 (time)
# OUTPUT: want x/y, dx/dy -> total input is n*4

i_col, o_col = [], []
colNames = ["m", "x", "y", "dx", "dy"]

for col in colNames:
    for n in range(1, nBodies+1):
        i_col.append(col+str(n))
        if col != "m":
            o_col.append(col+"f"+str(n))
i_col.append("t")
    
X1 = df.as_matrix(columns=i_col)
y1 = df.as_matrix(columns=o_col)

X_train,X_test,y_train,y_test = train_test_split(X1, y1, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.



(5152127, 16) (5152127, 28)
(1288032, 16) (1288032, 28)


In [93]:
loss_function = 'mse'
my_metrics = ['mae']

network = models.Sequential()
network.add(layers.Dense(X_train.shape[1], activation='relu', input_dim=X_train.shape[1]))
for i in range(10):
    network.add(layers.Dense(128,activation='relu'))
network.add(layers.Dense(y_train.shape[1],activation='linear'))
network.compile(optimizer='adam', loss=loss_function, metrics=my_metrics)
network.save_weights(workDir + '/weights/model_init_test.h5')

In [94]:
network.load_weights(workDir + '/weights/model_init_test.h5')
history = network.fit(X_train,y_train,
                      epochs=10,
                      batch_size=100,
                      validation_data=(X_test,y_test), 
                      verbose = 1)

Train on 5152127 samples, validate on 1288032 samples
Epoch 1/10
5152127/5152127 [==============================] - 77s 15us/step - loss: 0.4419 - modified_acc: 0.4078 - val_loss: 0.1627 - val_modified_acc: 0.3292
Epoch 2/10
5152127/5152127 [==============================] - 73s 14us/step - loss: 0.0758 - modified_acc: 0.2125 - val_loss: 0.0668 - val_modified_acc: 0.1944
Epoch 3/10
5152127/5152127 [==============================] - 73s 14us/step - loss: 0.0434 - modified_acc: 0.1596 - val_loss: 0.0338 - val_modified_acc: 0.1478
Epoch 4/10
5152127/5152127 [==============================] - 71s 14us/step - loss: 0.0331 - modified_acc: 0.1381 - val_loss: 0.0243 - val_modified_acc: 0.1228
Epoch 5/10
5152127/5152127 [==============================] - 72s 14us/step - loss: 0.0278 - modified_acc: 0.1249 - val_loss: 0.0337 - val_modified_acc: 0.1397
Epoch 6/10
5152127/5152127 [==============================] - 72s 14us/step - loss: 0.0246 - modified_acc: 0.1167 - val_loss: 0.0343 - val_modifie

In [1]:
# Plot training & validation accuracy values
plt.plot(history.history['mae'])
plt.plot(history.history['val_mae'])
plt.title('Model mae')
plt.ylabel('mae')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='best')
plt.savefig(workDir + 'model_mae.png')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='best')
plt.savefig(workDir + 'model_loss.png')
plt.show()

NameError: name 'plt' is not defined